In [1]:
# 導入資料
import numpy as np
import pandas as pd
df=pd.read_csv('LEC_SummerPlayoffs_2024.csv')
df.columns

Index(['Player', 'Role', 'Team', 'Opponent_Team', 'Opponent_Player', 'Date',
       'Round', 'Day', 'Patch', 'Stage', 'No_Game', 'all_Games', 'Format',
       'Game_of_day', 'Side', 'Time', 'Ban', 'Ban_Opponent', 'Pick',
       'Pick_Opponent', 'Champion', 'Champion_Opponent', 'Outcome',
       'Kills_Team', 'Turrets_Team', 'Dragon_Team', 'Baron_Team', 'Level',
       'Kills', 'Deaths', 'Assists', 'KDA', 'CS', 'CS in Team's Jungle',
       'CS in Enemy Jungle', 'CSM', 'Golds', 'GPM', 'GOLD%', 'Vision Score',
       'Wards placed', 'Wards destroyed', 'Control Wards Purchased',
       'Detector Wards Placed', 'VSPM', 'WPM', 'VWPM', 'WCPM', 'VS%',
       'Total damage to Champion', 'Physical Damage', 'Magic Damage',
       'True Damage', 'DPM', 'DMG%', 'K+A Per Minute', 'KP%', 'Solo kills',
       'Double kills', 'Triple kills', 'Quadra kills', 'Penta kills', 'GD@15',
       'CSD@15', 'XPD@15', 'LVLD@15', 'Objectives Stolen',
       'Damage dealt to turrets', 'Damage dealt to buildings', 

In [2]:
# 資料清洗
df_check=df['Outcome']
df_check=df_check[(df.index % 5 ==0)]
i=[i for i in range(84)]
df_check.index=i
df = df[['Side','Time','Turrets_Team',
    'Dragon_Team','Baron_Team',
    'CS','Golds','Vision Score','GD@15',
    'Objectives Stolen']]
g_num=[ i //5 for i in range(len(df))]
df.loc[:, 'Games']=g_num
side_dict={'Blue':0 ,'Red':1}
df['Side']=df['Side'].map(side_dict)
df=df[[
    'Side','Time','Turrets_Team',
    'Dragon_Team','Baron_Team',
    'CS','Golds','Vision Score','GD@15',
    'Objectives Stolen'
]].groupby(df['Games']).sum()
def time_cacu(x):
    timecacu = x[2:7].split(":")
    spend=int(timecacu[0])*60+int(timecacu[1])
    return spend
df['Time']=df['Time'].apply(time_cacu)
df.head()

,Side,Time,Turrets_Team,Dragon_Team,Baron_Team,CS,Golds,Vision Score,GD@15,Objectives Stolen
Games,,,,,,,,,,
0,0,1600,45,15,5,1047,53902,215,2138,0
1,5,1600,0,5,0,957,44209,267,-2138,0
2,0,1890,20,15,0,1046,52709,281,75,0
3,5,1890,45,10,10,1126,63526,260,-75,0
4,0,2160,25,5,0,1233,66570,310,242,0


In [3]:
# 取得mean與std 之後用於scale新的數據
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df=scaler.fit_transform(df)
mean=scaler.mean_
std_dev=scaler.scale_

In [4]:
# 建立從舊的df選場次的func
def choose(n):
    if n>83 or n<0:
        raise ValueError('n必須介於0~83之間 你的n=',n)
    else:
        target=df[n]
        target=target.reshape((1,10))
        print('Here is your target:\n', target)
        return target
# 對結果的更新由數字到結果
def outcome_changer(result):
    if result == 1:
        return 'Win'
    else:
        return 'Loss'
        
# 導入深度學習模型
from tensorflow import keras
dl_model=keras.models.load_model('dl_on_lol(ac88).h5')
# 導入二元回歸模型
import joblib
br_model=joblib.load('reg_on_lol(ac94)')

# 預測結果包裝成func
def predict(target):
    y_pred=dl_model.predict(target)
    print('Deep learning Predict Result: ',outcome_changer(y_pred))
    result=br_model.predict(target)
    print('Binary Regression Predict Result: ', outcome_changer(result))
    
# 給自訂新數據用 資料標準化的func
def data_scale(data):
    data-=mean
    data=data/std_dev
    return data

In [5]:
# 輸入n 檢驗當中的比賽場次
n=79
target=choose(n)
predict(target)
# 真實比賽的結果
print('Real-result:', df_check[n])

Here is your target:
 [[ 1.          0.39357619  0.75232905  0.48517032  0.18162732  0.63232743
   0.72412475  0.80195781 -1.1500041   2.7202941 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Deep learning Predict Result:  Loss
Binary Regression Predict Result:  Win
Real-result: Win


In [6]:
# 自己模擬比賽的數據進行預測
def pred(n):
    if type(n) != list or len(n) != 10:
        raise TypeError
        print('type must be list in 10 words')
    n=np.array([n])
    n=(n-mean)/std_dev
    print(n)
    result=predict(n)
    return result

In [7]:
# 自製隊伍數據進行預測
# 看起來有點複雜，因為除了Side 和 time 其他的單位都是5個人的總和...
"""
Side Time Turrets_Team Dragon_Team Baron_Team CS Golds Vision_Score GD@15	Objectives_Stolen
"""
t=[0,1800,15,25,10,1200,60000,280,-1800,0]
pred(t)
r=[5,1600,45,15,5,1047,53902,215,2138,0]
pred(r)

[[-1.         -0.64308864 -1.12476918  1.96714513  1.35521923  0.32180773
  -0.2398501  -0.32391054 -0.70384474 -0.36760731]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Deep learning Predict Result:  Loss
Binary Regression Predict Result:  Win
[[ 1.         -1.2371659   0.75232905  0.48517032  0.18162732 -0.52657646
  -0.76194963 -1.26213416  0.83601114 -0.36760731]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Deep learning Predict Result:  Loss
Binary Regression Predict Result:  Win


In [8]:
# 把上面的預測簡化成基本的func
question=['請輸入隊伍紅藍方，藍方=0。紅方=1',
          '請輸入比賽用時(請輸入總用時秒數 ex.20分30秒輸入1500)',
          '請輸入隊伍推塔的數量(介於0~9之間)',
          '請輸入隊伍集殺的小龍的數量(0~6之間)',
          '請輸入隊伍集殺的巴龍的數量(0~3之間)',
          '請輸入隊伍5人的cs數總和(700~3500)',
          '請輸入隊伍5人的Gold數總和(25000~100000)',
          '請輸入隊伍5人的視野分數總和(70~1000)',
          '請輸入隊伍5人的15分鐘經濟差(-10000~10000)',
          '請輸入隊伍5人偷取中立物件(0~1)'
    ]

def lol_win_predict():
    count = 0
    q=[]
    while count != 10:
        print(question[count],": \n")
        i=int(input())*5
        if count == 1 or 4<count<9:
            i = i/5
        q.append(i)
        count+=1
    print('你輸入的數值已轉換完畢\n',q)
    pred(q)

In [ ]:
lol_win_predict()

請輸入隊伍紅藍方，藍方=0。紅方=1 : 

